In [120]:
import math 
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [121]:
class Value:

    def __init__(self,data,_clildren=(),_op='',label=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self.label = label
        self._prev = set(_clildren)
        self._op = _op

    def __repr__(self):
        return f'Value(data={self.data})'
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data, (self,other), '+')
        def _backward():
            self.grad += 1 * out.grad
            other.grad += 1 * out.grad
        out._backward = _backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data * other.data, (self,other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self,other):
        return self * (other**-1)
    
    def __pow__(self,other):
        assert isinstance(other, (int,float))
        out = Value(self.data**other, (self,),f'**{other}')
        def _backward():
            self.grad = other * self.data**(other-1) * out.grad
        out._backward = _backward
        return out
    
    def __sub__(self,other):
        return self + (-other)
    
    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) -1)/(math.exp(2*x)+1)
        out = Value(t,(self,),'tanh')
        def _backward():
            self.grad = (1- t**2) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        e = (math.exp(x))
        out = Value(e,(self,),'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def backward(self):
        self.grad = 1
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
            return topo
        build_topo(self)
        for node in reversed(topo):
            node._backward()
    

In [122]:
# x1 = Value(2,label='x1')
# x2 = Value(0,label='x2')

# w1 = Value(-3,label='w1')
# w2 = Value(1,label='w2')

# b = Value(6.8814)

# x1w1 = x1*w1; x1w1.label = 'x1w1'
# x2w2 = x2*w2; x2w2.label = 'x2w2'

# x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1 + x2w2'

# n = x1w1x2w2 + b; n.label= 'n'
# o = n.tanh(); o.label = 'tanh'


In [123]:
# o.backward()

In [124]:
# from graphviz import Digraph

# def trace(root):
#   # builds a set of all nodes and edges in a graph
#   nodes, edges = set(), set()
#   def build(v):
#     if v not in nodes:
#       nodes.add(v)
#       for child in v._prev:
#         edges.add((child, v))
#         build(child)
#   build(root)
#   return nodes, edges

# def draw_dot(root):
#   dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
#   nodes, edges = trace(root)
#   for n in nodes:
#     uid = str(id(n))
#     # for any value in the graph, create a rectangular ('record') node for it
#     dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
#     if n._op:
#       # if this value is a result of some operation, create an op node for it
#       dot.node(name = uid + n._op, label = n._op)
#       # and connect this node to it
#       dot.edge(uid + n._op, uid)

#   for n1, n2 in edges:
#     # connect n1 to the op node of n2
#     dot.edge(str(id(n1)), str(id(n2)) + n2._op)

#   return dot

In [125]:
class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    def __call__(self,x):
        act = sum((wi* xi for wi,xi in zip(self.w,x)), self.b)
        out = act.tanh()
        return out
    def parameters(self):
        return self.w + [self.b]

In [126]:
class Layer:
    def __init__(self,nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self,x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out)==1 else out
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

In [127]:
class MLP:
    def __init__(self,nin,nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [135]:
n = MLP(3,[4,4,1])

In [136]:
xs = [[2,-3,0],
      [-1,1,5,1],
      [0.5,2.5,1.5],
      [1,2,-0.5]]

ys = [1,-1,-1,1]

In [137]:
for _ in range(20):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    for p in n.parameters():
        p.data -= 0.1*p.grad
    print(loss.data)

6.081498336035429
3.664158200908692
2.6975700944410272
2.406272235977596
2.036897355654713
1.5562971411276125
1.0533068671692984
0.7297970368368265
0.5401565636896661
0.4035490008703062
0.21247996928026075
0.15165410603241958
0.12625948927741237
0.10771690778533427
0.09354128544565099
0.08240243256321977
0.07345088361289026
0.06612059321475527
0.06002148937847932
0.054876965447030766


In [138]:
ypred

[Value(data=0.9843621094221031),
 Value(data=-0.8991496632333286),
 Value(data=-0.8706029632791225),
 Value(data=0.8335126482665746)]

In [139]:
for data in ypred:
    if data.data < 0:
        data.data = -1 
    else:
        data.data = 1
    print(data.data)

1
-1
-1
1
